In [21]:
from google.cloud import storage
from pathlib import Path
from google.cloud import storage
import json
from google.cloud import vision


In [22]:
project_root_dir = Path("/Users/phapman/Desktop/tnh-scholar")

In [23]:
def upload_to_gcs(pdf_path: Path, bucket_name: str, gcs_destination: str) -> str:
    """
    Uploads a local PDF file to Google Cloud Storage.
    
    Parameters:
        pdf_path (Path): Path to the local PDF.
        bucket_name (str): GCS bucket name.
        gcs_destination (str): Path in the GCS bucket.
    
    Returns:
        str: The GCS URI of the uploaded PDF.
    """
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(gcs_destination)
    blob.upload_from_filename(str(pdf_path))
    gcs_uri = f"gs://{bucket_name}/{gcs_destination}"
    print(f"Uploaded {pdf_path} to {gcs_uri}")
    return gcs_uri

In [24]:
def batch_process_pdf(gcs_uri: str, output_gcs_uri: str) -> str:
    """
    Sends a batch processing request to Google Vision API for document text detection.
    
    Parameters:
        gcs_uri (str): URI of the PDF in GCS.
        output_gcs_uri (str): GCS URI for storing the result.
    
    Returns:
        str: The operation name for tracking the job.
    """
    client = vision.ImageAnnotatorClient()

    # Input configuration
    input_config = vision.InputConfig(
        gcs_source=vision.GcsSource(uri=gcs_uri),
        mime_type="application/pdf"
    )

    # Output configuration for asynchronous results
    output_config = vision.GcsDestination(uri=output_gcs_uri)

    # Feature type: DOCUMENT_TEXT_DETECTION
    features = [vision.Feature(type=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)]

    # File request (Corrected: `output_config` goes into `AsyncAnnotateFileRequest`)
    async_request = vision.AsyncAnnotateFileRequest(
        input_config=input_config,
        features=features,
        output_config=vision.OutputConfig(gcs_destination=output_config, batch_size=1)
    )

    # Async batch request
    operation = client.async_batch_annotate_files(requests=[async_request])
    print(f"Started batch operation: {operation.operation.name}")
    return operation.operation.name

In [51]:
import time

def poll_operation_status_with_retry(operation_name: str, poll_interval: int = 10, max_retries: int = 30) -> None:
    """
    Polls the status of an asynchronous batch operation with retries.
    
    Parameters:
        operation_name (str): Name of the operation to check.
        poll_interval (int): Time (in seconds) to wait between retries.
        max_retries (int): Maximum number of retries before giving up.
    """
    client = vision.ImageAnnotatorClient()

    for attempt in range(max_retries):
        operation = client.transport.operations_client.get_operation(operation_name)
        
        if operation.done:
            if operation.HasField('error'):
                print(f"Operation failed with error: {operation.error.message}")
                return operation
            elif operation.HasField('response'):
                print("Operation completed successfully.")
                return True # Exit the loop upon success or failure
            else:
                print("Unknown operation status.")
                return operation
        else:
            print(f"Attempt {attempt + 1}/{max_retries}: Operation still in progress...")
            time.sleep(poll_interval)

    print("Polling timed out. The operation may still be in progress.")

In [ ]:
def download_and_parse_results(output_gcs_uri: str, local_output_dir: Path) -> None:
    """
    Downloads and parses batch processing results from GCS.
    
    Parameters:
        output_gcs_uri (str): URI of the GCS folder containing results.
        local_output_dir (Path): Local directory to save the results.
    """
    storage_client = storage.Client()
    bucket_name, prefix = output_gcs_uri.replace("gs://", "").split("/", 1)
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)
    pages = []

    local_output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Downloading results to {local_output_dir}...")

    for blob in blobs:
        local_file_path = local_output_dir / blob.name.split("/")[-1]
        blob.download_to_filename(str(local_file_path))
        print(f"Downloaded: {blob.name}")

        # Parse JSON output
        with open(local_file_path, "r") as f:
            result = json.load(f)
            for page in result.get("responses", []):
                pages.append(page.get("fullTextAnnotation", {}).get("text", ""))
    
    return pages

In [120]:
def download_results(output_gcs_uri: str):
    """
    Downloads and parses batch processing results from GCS directly into memory.

    Parameters:
        output_gcs_uri (str): URI of the GCS folder containing results.

    Returns:
        List[str]: A list of texts extracted from the results.
    """
    from google.cloud import storage
    import json

    storage_client = storage.Client()
    bucket_name, prefix = output_gcs_uri.replace("gs://", "").split("/", 1)
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)
    pages = []

    print("Downloading and parsing results in memory...")

    for blob in blobs:
        print(f"Processing: {blob.name}")
        
        # Read blob content directly into memory
        content = blob.download_as_bytes()
        
        # Parse JSON from memory
        result = json.loads(content)
        for page in result.get("responses", []):
            text = page.get("fullTextAnnotation", {}).get("text", "")
            if text:
                pages.append(text)

    return result, pages

In [117]:
pdf_dir_path = project_root_dir / "data_processing/PDF/Phat_Giao_journals"
pdf_path = pdf_dir_path / "phat-giao-viet-nam-1956-01.pdf"

In [118]:
bucket_name = "test-bucket-tnh-translation"
gcs_destination = "vietnamese_docs/vietnamese.pdf"
output_gcs_uri = "gs://test-bucket-tnh-translation/vietnamese_docs/output/"
local_output_dir = Path("./local_results")

In [ ]:
# Step 1: Upload PDF to GCS
# gcs_uri = upload_to_gcs(pdf_path, bucket_name, gcs_destination)

In [ ]:
# Step 2: Start batch processing
# operation_name = batch_process_pdf(gcs_uri, output_gcs_uri)

I0000 00:00:1732417874.542762 3887049 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Started batch operation: projects/tnh-scholar/operations/3b9993cbe4400cdc


In [52]:
# Step 3: Poll the operation status (repeat until done)
result = poll_operation_status_with_retry(operation_name)

I0000 00:00:1732425449.701308 3887049 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Operation completed successfully.


In [119]:
operation_name

'projects/tnh-scholar/operations/3b9993cbe4400cdc'

In [124]:
# Step 4: Download and parse results
full_data, pages = download_results(output_gcs_uri)

Processing: vietnamese_docs/output/output-1-to-1.json
Processing: vietnamese_docs/output/output-10-to-10.json
Processing: vietnamese_docs/output/output-11-to-11.json
Processing: vietnamese_docs/output/output-12-to-12.json
Processing: vietnamese_docs/output/output-13-to-13.json
Processing: vietnamese_docs/output/output-14-to-14.json
Processing: vietnamese_docs/output/output-15-to-15.json
Processing: vietnamese_docs/output/output-16-to-16.json
Processing: vietnamese_docs/output/output-17-to-17.json
Processing: vietnamese_docs/output/output-18-to-18.json
Processing: vietnamese_docs/output/output-19-to-19.json
Processing: vietnamese_docs/output/output-2-to-2.json
Processing: vietnamese_docs/output/output-20-to-20.json
Processing: vietnamese_docs/output/output-21-to-21.json
Processing: vietnamese_docs/output/output-22-to-22.json
Processing: vietnamese_docs/output/output-23-to-23.json
Processing: vietnamese_docs/output/output-24-to-24.json
Processing: vietnamese_docs/output/output-25-to-25.j

In [125]:
pages[0]

'PHẬT GIÁO\nVIET-NAM\nNGUYỆT-SAN\nSỐ 1 RA NGÀY 15 THÁNG 8 BÍNH THÂN THE\nTV\nTỔNG. HỘI PHẬT - GIÁO VIỆT - NAM XUẤT.\nHUỆ QUANG'

In [130]:
full_data.keys()

dict_keys(['inputConfig', 'responses'])

In [133]:
responses = full_data['responses']

In [137]:
responses[0]['text_annotation']

KeyError: 'text_annotation'

In [62]:
def parse_ocr_json(json_path):
    """
    Parses a Google Vision OCR JSON output file to extract blocks and paragraphs with their bounding boxes.
    Navigates through the hierarchy starting from the 'responses' key.

    Args:
        json_path (str): Path to the Google Vision OCR output JSON file.

    Returns:
        list: A list of dictionaries, each representing a block with its text and paragraphs. 
              Each block contains:
                - 'block_text': Full text of the block
                - 'bounding_box': Bounding box of the block
                - 'paragraphs': List of paragraphs, each containing:
                    - 'paragraph_text': Full text of the paragraph
                    - 'bounding_box': Bounding box of the paragraph
    """
    import json

    with open(json_path, 'r') as f:
        ocr_data = json.load(f)

    parsed_data = []

    # Navigate to 'responses' -> 'fullTextAnnotation' -> 'pages' -> 'blocks'
    responses = ocr_data.get('responses', [])
    for response in responses:
        full_text_annotation = response.get('fullTextAnnotation', {})
        pages = full_text_annotation.get('pages', [])
        
        for page in pages:
            for block in page.get('blocks', []):
                block_text = []
                block_bounding_box = block.get('boundingBox', {})
                paragraphs = []

                for paragraph in block.get('paragraphs', []):
                    paragraph_text = []
                    paragraph_bounding_box = paragraph.get('boundingBox', {})

                    for word in paragraph.get('words', []):
                        word_text = ''.join(symbol.get('text', '') for symbol in word.get('symbols', []))
                        paragraph_text.append(word_text)

                    paragraphs.append({
                        'paragraph_text': ' '.join(paragraph_text),
                        'bounding_box': paragraph_bounding_box
                    })
                    block_text.extend(paragraph_text)

                parsed_data.append({
                    'block_text': ' '.join(block_text),
                    'bounding_box': block_bounding_box,
                    'paragraphs': paragraphs
                })

    return parsed_data


In [102]:
def get_origin_point(element):
    """
    Extracts the origin point (first vertex) of the bounding box.

    Args:
        element (dict): A block or paragraph dictionary containing a 'bounding_box' key.

    Returns:
        tuple: The (x, y) origin point of the bounding box, or None if not found.
    """
 
    bounding_box = element.get("bounding_box", {})
    vertices = bounding_box.get("normalizedVertices", [])

    if vertices and len(vertices) > 0:
        point = vertices[0].get("x", 0), vertices[0].get("y", 0)
        return point
    return None

In [106]:
def build_xml_from_blocks(parsed_data, include_attributes=[]):
    """
    Converts parsed block and paragraph data into XML-structured text with optional attributes.

    Args:
        parsed_data (list): A list of dictionaries, each representing a block with text, bounding boxes, 
                            and paragraphs (output from parse_ocr_json).
        include_attributes (list): A list of attributes to include in the tags. Example: ["bounding_box", "origin_point"].

    Returns:
        str: XML-structured text representing the data.
    """
    from xml.etree.ElementTree import Element, SubElement, tostring
    from xml.dom.minidom import parseString

    assert isinstance(include_attributes, list)

    # Define a mapping of attribute names to helper functions
    attribute_extractors = {
        "origin_point": get_origin_point,
    }

    # Root element
    root = Element("document")

    # Process each block
    for block in parsed_data:
        # Prepare attributes for the block
        block_attributes = {
            key: str(attribute_extractors[key](block))
            for key in include_attributes if key in attribute_extractors
        }
        block_element = SubElement(root, "block", attrib=block_attributes)

        # Add paragraphs within the block
        for paragraph in block.get("paragraphs", []):
            # Prepare attributes for the paragraph
            paragraph_attributes = {
                key: str(attribute_extractors[key](paragraph))
                for key in include_attributes if key in attribute_extractors
            }
            paragraph_element = SubElement(
                block_element, "p", attrib=paragraph_attributes
            )
            paragraph_element.text = paragraph.get("paragraph_text", "")

    # Generate pretty XML string
    rough_string = tostring(root, encoding="unicode")
    pretty_xml = parseString(rough_string).toprettyxml(indent="  ")
    return pretty_xml

In [82]:
test_file = local_output_dir / "output-1-to-1.json"

In [83]:
test_file.exists()

True

In [109]:
result = parse_ocr_json(local_output_dir / "output-3-to-3.json")

In [110]:
result

[{'block_text': 'IPIHIAT - GIAO VIET = NAM',
  'bounding_box': {'normalizedVertices': [{'x': 0.11673152, 'y': 0.11235955},
    {'x': 0.58236057, 'y': 0.10976664},
    {'x': 0.58365756, 'y': 0.22212619},
    {'x': 0.11802854, 'y': 0.2238548}]},
  'paragraphs': [{'paragraph_text': 'IPIHIAT - GIAO',
    'bounding_box': {'normalizedVertices': [{'x': 0.11673152, 'y': 0.11495246},
      {'x': 0.5797665, 'y': 0.110630944},
      {'x': 0.58106357, 'y': 0.15384616},
      {'x': 0.11802854, 'y': 0.15816768}]}},
   {'paragraph_text': 'VIET = NAM',
    'bounding_box': {'normalizedVertices': [{'x': 0.11932555, 'y': 0.19187555},
      {'x': 0.58236057, 'y': 0.19187555},
      {'x': 0.58236057, 'y': 0.22212619},
      {'x': 0.11932555, 'y': 0.22212619}]}}]},
 {'block_text': 'NG.VI',
  'bounding_box': {'normalizedVertices': [{'x': 0.6757458, 'y': 0.23336214},
    {'x': 0.8715953},
    {'x': 0.95071334, 'y': 0.027657736},
    {'x': 0.7548638, 'y': 0.26274848}]},
  'paragraphs': [{'paragraph_text': 'NG.

In [113]:
print(build_xml_from_blocks(result, include_attributes=["origin_point"]))

<?xml version="1.0" ?>
<document>
  <block origin_point="(0.11673152, 0.11235955)">
    <p origin_point="(0.11673152, 0.11495246)">IPIHIAT - GIAO</p>
    <p origin_point="(0.11932555, 0.19187555)">VIET = NAM</p>
  </block>
  <block origin_point="(0.6757458, 0.23336214)">
    <p origin_point="(0.6757458, 0.23336214)">NG.VI</p>
  </block>
  <block origin_point="(0.1154345, 0.25583404)">
    <p origin_point="(0.17898832, 0.25583404)">TỪ NGÀY 10 Tỳ - Ni - Đa - Lưu - Chi sang nước ta đến nay ,</p>
    <p origin_point="(0.1154345, 0.29299915)">kề ra đã đến mười lăm thế kỷ . Phật - Giáo đã ở lại cùng chúng ta một ngàn năm trăm năm , và đã cùng dân tộc Việt - Nam chịu chung bao nhiêu thắng - trầm vinh nhục ,</p>
    <p origin_point="(0.16990921, 0.3621435)">Phật - Giáo Việt - Nam quả là một nền Phật - Giáo dân tộc .</p>
  </block>
  <block origin_point="(0.11673152, 0.7372515)">
    <p origin_point="(0.11673152, 0.7372515)">PHẬT - GIÁO VIỆT NAM</p>
  </block>
  <block origin_point="(0.5149157,

In [105]:
0.9628349 * 1157

1113.9999793

In [114]:
print(build_xml_from_blocks(result))

<?xml version="1.0" ?>
<document>
  <block>
    <p>IPIHIAT - GIAO</p>
    <p>VIET = NAM</p>
  </block>
  <block>
    <p>NG.VI</p>
  </block>
  <block>
    <p>TỪ NGÀY 10 Tỳ - Ni - Đa - Lưu - Chi sang nước ta đến nay ,</p>
    <p>kề ra đã đến mười lăm thế kỷ . Phật - Giáo đã ở lại cùng chúng ta một ngàn năm trăm năm , và đã cùng dân tộc Việt - Nam chịu chung bao nhiêu thắng - trầm vinh nhục ,</p>
    <p>Phật - Giáo Việt - Nam quả là một nền Phật - Giáo dân tộc .</p>
  </block>
  <block>
    <p>PHẬT - GIÁO VIỆT NAM</p>
  </block>
  <block>
    <p>Phật - Giáo Việt Nam không</p>
    <p>phải chỉ là một tồn - giáo tin ngưỡng mà bất cứ thời nào , ở đâu , cũng chỉ biết có sứ - mạng của lồn - giáo tín ngưỡng . Không 1 Ở bất cứ nước nào trên thế giới cũng vậy , khi bước chân đến , Đạo Phật cũng thích nghỉ ngay với phong - lục , khí hậu , nhân tính đề biển thành một lối sống cho quần chúng . Ở Việt Nam cũng thể . Phật - Giáo đã hòa hợp trong cá tính dân tộc ta , đã cùng dân tộc ta xây dựng một văn

<?xml version="1.0" ?>
<document>
  <block>
    <p>IPIHIAT - GIAO</p>
    <p>VIET = NAM</p>
  </block>
  <block>
    <p>NG.VI</p>
  </block>
  <block>
    <p>TỪ NGÀY 10 Tỳ - Ni - Đa - Lưu - Chi sang nước ta đến nay ,</p>
    <p>kề ra đã đến mười lăm thế kỷ . Phật - Giáo đã ở lại cùng chúng ta một ngàn năm trăm năm , và đã cùng dân tộc Việt - Nam chịu chung bao nhiêu thắng - trầm vinh nhục ,</p>
    <p>Phật - Giáo Việt - Nam quả là một nền Phật - Giáo dân tộc .</p>
  </block>
  <block>
    <p>PHẬT - GIÁO VIỆT NAM</p>
  </block>
  <block>
    <p>Phật - Giáo Việt Nam không</p>
    <p>phải chỉ là một tồn - giáo tin ngưỡng mà bất cứ thời nào , ở đâu , cũng chỉ biết có sứ - mạng của lồn - giáo tín ngưỡng . Không 1 Ở bất cứ nước nào trên thế giới cũng vậy , khi bước chân đến , Đạo Phật cũng thích nghỉ ngay với phong - lục , khí hậu , nhân tính đề biển thành một lối sống cho quần chúng . Ở Việt Nam cũng thể . Phật - Giáo đã hòa hợp trong cá tính dân tộc ta , đã cùng dân tộc ta xây dựng một văn hóa quốc gia độc - lập .</p>
  </block>
  <block>
    <p>3</p>
  </block>
</document>

